# Atividade II

Crie uma tabela com os dados de Retorno, Volatilidade, Alfa, Beta, Sharpe e Sortino para cada um dos portfólios.

In [45]:
import pandas as pd
import numpy as np

import config as cfg
import helpers as hlp

In [46]:
# Lendo a lisrta de portfólios
df_portfolio_list = pd.read_csv(f'{cfg.PATH_RESULTADOS}/portfolio_files.csv')

df_portfolio_list.head()

,Portfolio,File Name
0,Blue Chips,blue_chips.parquet
1,Mid Caps,mid_caps.parquet
2,Small Caps,small_caps.parquet
3,Micro Caps,micro_caps.parquet
4,High Return,high_return.parquet


## 1. Calculando as variáveis para o portfólio Blue Chips

In [47]:
# Recuperando o arquivo
portfolio_name = df_portfolio_list['Portfolio'].iloc[0]
file_name = df_portfolio_list['File Name'].iloc[0]

df_blue_chips = pd.read_parquet(f"{cfg.PATH_CALCULATED_PORTFOLIO_DIR}/{file_name}", columns=['TOTAL AMOUNT']).rename(columns={'TOTAL AMOUNT': portfolio_name})

df_blue_chips.head()

,Blue Chips
Data,
2019-12-30,10000000.05
2020-01-02,10268933.99
2020-01-03,10229799.50
2020-01-06,10116688.58
2020-01-07,10155429.40


In [48]:
def log_return(df, column):
    return np.log(df[column]/df[column].shift(1))

In [49]:
LOG_COLUMN_NAME = "Log Return"

df_blue_chips[LOG_COLUMN_NAME]=log_return(df_blue_chips, portfolio_name)

df_blue_chips[LOG_COLUMN_NAME].fillna(0, inplace=True)

In [50]:
df_blue_chips[f'{LOG_COLUMN_NAME} from 5D'] = df_blue_chips[LOG_COLUMN_NAME].rolling(5).sum()
df_blue_chips[f'{LOG_COLUMN_NAME} from 21D'] = df_blue_chips[LOG_COLUMN_NAME].rolling(21).sum()
df_blue_chips[f'{LOG_COLUMN_NAME} from 252D'] = df_blue_chips[LOG_COLUMN_NAME].rolling(252).sum()

df_blue_chips[f'Acum. {LOG_COLUMN_NAME}'] = df_blue_chips[LOG_COLUMN_NAME].cumsum()

# A volatilidade anualizada
df_blue_chips["Vol"] = df_blue_chips[LOG_COLUMN_NAME].rolling(21).std()*np.sqrt(252)

In [51]:
df_blue_chips.tail()

,Blue Chips,Log Return,Log Return from 5D,Log Return from 21D,Log Return from 252D,Acum. Log Return,Vol
Data,,,,,,,
2023-03-27,11369455.34,0.009320,-0.007184,-0.041779,-0.125607,0.128345,0.162762
2023-03-28,11486797.21,0.010268,0.002770,-0.029285,-0.110500,0.138613,0.168197
2023-03-29,11558324.82,0.006208,0.016817,-0.024672,-0.106580,0.144821,0.169980
2023-03-30,11738135.77,0.015437,0.050427,-0.003506,-0.098746,0.160258,0.178438
2023-03-31,11524379.66,-0.018378,0.022855,-0.010750,-0.120383,0.141880,0.185666


In [52]:
df_prelast = pd.read_parquet(cfg.PATH_PRELAST_PARQUET, columns=['.BVSP', 'DI_INDEX'])

In [53]:
df_prelast.head()

,.BVSP,DI_INDEX
0,115645.34,688179.09
1,118573.10,688414.31
2,117706.66,688531.95
3,116877.92,688649.61
4,116661.94,688767.29
